In [ ]:
import pandas as pd
import plotly.express as px
import math
import json
import numpy as np

In [ ]:
def column_metrics(df):
    metrics = {
        'Mínimo': df.min(),
        'Mediana': df.median(),
        'Média': df.mean(),
        'Máximo': df.max(),
        'Nulos': df.isnull().sum(),
        'Não-Nulos': df.notnull().sum()
    }
    result_df = pd.DataFrame(metrics)
    return result_df

In [ ]:
df_1 = _deepnote_execute_sql('SELECT *\nFROM \'/work/Sample Data/export_sensedata_clientes_2023-08-03 (2).csv\'\nwhere status = \'Ativo\'\nand not Segmento like \'Light\';', 'SQL_DEEPNOTE_DATAFRAME_SQL', audit_sql_comment='', sql_cache_mode='cache_disabled')
df_1

,ID Original,Sense Score,MRR,Status,Segmento,NPS,Chamados acima do SLA,Qtd criterios ativação,Chamados acima do SLA_1,Boletos Emitidos,...,Variação de MRR 1M,Quantidade de contratos,Quantidade de licenças,Boletos Emitidos_1,Licenças,Engagement Score,Títulos Vencidos,Usuários Ativos,LT (meses),ID Sensedata
0,5302623783485440,90.131579,317.000000,Ativo,Smart,NaN,0,1,0,0.0,...,NaN,0,0,0.0,500,73.684211,0,1,0,100357
1,5401690169737216,94.078947,317.000000,Ativo,Smart,NaN,0,2,0,0.0,...,NaN,0,0,0.0,500,84.210526,0,1,0,100356
2,5856697243467776,100.000000,557.000000,Ativo,Company,NaN,0,1,0,0.0,...,1.0,0,0,0.0,1000,100.000000,0,10,0,100352
3,5894870661136384,82.236842,197.000000,Ativo,Up,NaN,0,2,0,0.0,...,1.0,0,0,0.0,150,52.631579,0,2,0,100348
4,5154634251436032,76.315789,197.000000,Ativo,Up,NaN,0,1,0,0.0,...,1.0,0,0,0.0,150,36.842105,0,2,0,100328
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14217,5721310499962880,96.842105,391.810000,Ativo,Pro,100.0,0,5,0,0.0,...,1.0,8,515,0.0,3,89.473684,0,3,117,12
14218,5665395394478080,81.052632,561.600000,Ativo,Starter Multi,100.0,0,5,0,0.0,...,1.0,8,1014,0.0,1009,36.842105,0,5,117,10
14219,5747687638958080,20.000000,143.120000,Ativo,Starter Solo,100.0,0,3,0,0.0,...,1.0,9,506,0.0,501,0.000000,0,1,118,8
14220,5767072705413120,88.771930,222.750000,Ativo,Smart,67.0,0,4,0,0.0,...,1.0,0,0,0.0,500,73.684211,0,2,119,7


In [ ]:
colunas_com_outliers = [
    'MRR',
   # '% Usuários Ativos (Semana)',
    '% Créditos Utilizados',
    'Variação Engagement 30x30',
    'Variação Engagement 7x7']

In [ ]:
df_treated = df_1
for column in df_treated.columns:
    if (df_treated[f"{column}"].dtype) in ["int64", "float64"] and column in colunas_com_outliers:
        Q1 = df_treated[f"{column}"].quantile(0.25)
        Q3 = df_treated[f"{column}"].quantile(0.75)
        IQR = Q3 - Q1
        upper = Q3 + 1.5 * IQR
        lower = Q1 - 1.5 * IQR
        df_treated[f"{column}"] = df_1[f"{column}"].apply(lambda x:  min(x, upper) if (x) else None)
        df_treated[f"{column}"] = df_1[f"{column}"].apply(lambda x: max(x, lower) if (x) else None)
               # caso o valor da pontuação do cliente seja superior ao da cerca superior, o valor da cerca é considerado.
    else:
        pass

In [ ]:
#df_1.to_csv('kpi_analysis/treated.csv')

In [ ]:
#df_1 = pd.read_csv('kpi_analysis/treated.csv', index_col=0)

In [ ]:
metrics = column_metrics(df_1)
metrics

,Mínimo,Mediana,Média,Máximo,Nulos,Não-Nulos
% Créditos Utilizados,0.032,5.320000e+01,5.344152e+01,150.383333,441,13781
% Usuários Ativos (Semana),0.621118,1.428571e+01,1.237847e+01,200.0,1443,12779
Andamentos contratados,40.0,5.000000e+02,6.092222e+02,50000.0,3,14219
Boletos Emitidos,0.0,0.000000e+00,5.487554e+01,15111.0,0,14222
Boletos Emitidos_1,0.0,0.000000e+00,5.487554e+01,15111.0,0,14222
Boletos Recebidos,0.0,0.000000e+00,2.520243e+01,10241.0,0,14222
Chamados acima do SLA,0,0.000000e+00,0.000000e+00,0,0,14222
Chamados acima do SLA_1,0,0.000000e+00,0.000000e+00,0,0,14222
Critérios de Ativação,0.0,1.000000e+02,8.713542e+01,100.0,0,14222
Dias Public. Tratadas (Semana),0.0,1.000000e+00,1.549360e+00,7.0,0,14222


In [ ]:
SenseScoreMinimo = 0

In [ ]:
EngScoreMinimo = 0

In [ ]:
LTMinimo = 0

In [ ]:
Segmento = 'Todos'

In [ ]:
if Segmento == 'Todos':
    df_filtered = df_treated
else:
    df_filtered = df_treated[df_treated['Segmento'] == Segmento]
df_filtered = df_filtered.drop(columns=['ID Original','Status','Segmento'])
df_filtered = df_filtered[df_treated['Sense Score'] >= SenseScoreMinimo]
df_filtered = df_filtered[df_treated['Engagement Score'] >= EngScoreMinimo]
df_filtered = df_filtered[df_treated['LT (meses)'] >= LTMinimo]
df_filtered.sort_index()

,Sense Score,MRR,NPS,Chamados acima do SLA,Qtd criterios ativação,Chamados acima do SLA_1,Boletos Emitidos,Boletos Recebidos,Variação Engagement 7x7,Qtd títulos pagos após vencimento,...,Variação de MRR 1M,Quantidade de contratos,Quantidade de licenças,Boletos Emitidos_1,Licenças,Engagement Score,Títulos Vencidos,Usuários Ativos,LT (meses),ID Sensedata
0,90.131579,317.000000,NaN,0,1,0,0.0,0.0,NaN,NaN,...,NaN,0,0,0.0,500,73.684211,0,1,0,100357
1,94.078947,317.000000,NaN,0,2,0,0.0,0.0,NaN,NaN,...,NaN,0,0,0.0,500,84.210526,0,1,0,100356
2,100.000000,557.000000,NaN,0,1,0,0.0,0.0,NaN,NaN,...,1.0,0,0,0.0,1000,100.000000,0,10,0,100352
3,82.236842,197.000000,NaN,0,2,0,0.0,0.0,NaN,NaN,...,1.0,0,0,0.0,150,52.631579,0,2,0,100348
4,76.315789,197.000000,NaN,0,1,0,0.0,0.0,NaN,NaN,...,1.0,0,0,0.0,150,36.842105,0,2,0,100328
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14217,96.842105,391.810000,100.0,0,5,0,0.0,0.0,NaN,55.0,...,1.0,8,515,0.0,3,89.473684,0,3,117,12
14218,81.052632,561.600000,100.0,0,5,0,0.0,0.0,9.756098,19.0,...,1.0,8,1014,0.0,1009,36.842105,0,5,117,10
14219,20.000000,143.120000,100.0,0,3,0,0.0,0.0,-17.600964,80.0,...,1.0,9,506,0.0,501,0.000000,0,1,118,8
14220,88.771930,222.750000,67.0,0,4,0,0.0,0.0,-8.181818,63.0,...,1.0,0,0,0.0,500,73.684211,0,2,119,7


In [ ]:
column = 'Sense Score'
fig = px.violin(df_filtered, y=f"{column}", title=f'Distribuição de {column}')
fig.show()

In [ ]:
column = 'MRR'
fig = px.violin(df_filtered, y=f"{column}", title=f'Distribuição de {column}')
fig.show()

In [ ]:
column = 'Dias sem login'
fig = px.violin(df_filtered, y=f"{column}", title=f'Distribuição de {column}')
fig.show()

In [ ]:
column = 'Qtd criterios ativação'
fig = px.histogram(df_filtered, x=f"{column}", title=f'Distribuição de {column}')
fig.show()

In [ ]:
column = 'Dias Public. Tratadas (Semana)'
fig = px.histogram(df_filtered, x=f"{column}", title=f'Distribuição de {column}')
fig.show()

In [ ]:
column = 'Engagement Score'
fig = px.violin(df_filtered, y=f"{column}", title=f'Distribuição de {column}')
fig.show()

In [ ]:
column = '% Créditos Utilizados'
fig = px.violin(df_filtered, y=f"{column}", title=f'Distribuição de {column}')
fig.show()

In [ ]:
column = '% Usuários Ativos (Semana)'
fig = px.violin(df_filtered, y=f"{column}", title=f'Distribuição de {column}')
fig.show()

In [ ]:
column = 'Variação de MRR 1M'
fig = px.violin(df_filtered, y=f"{column}", title=f'Distribuição de {column}')
fig.show()

In [ ]:
column = 'Variação Engagement 30x30'
fig = px.histogram(df_filtered, x=f"{column}", title=f'Distribuição de {column}')
fig.show()

In [ ]:
column = 'Variação Engagement 7x7'
fig = px.violin(df_filtered, y=f"{column}", title=f'Distribuição de {column}')
fig.show()

In [ ]:
results_1 = pd.read_csv('/work/Results/results_v1.csv', index_col=0)
results_2 = pd.read_csv('/work/Results/results_v2.csv', index_col=0)
results_3 = pd.read_csv('/work/Results/results_v3.csv', index_col=0)
results_4 = pd.read_csv('/work/Results/results_v4.csv', index_col=0)
results_5 = pd.read_csv('/work/Results/results_v5.csv', index_col=0)

dfs = [results_1,results_2,results_3,results_4,results_5]
df_comparison = pd.concat(dfs, axis=1, join='inner')

In [ ]:
df_2 = _deepnote_execute_sql('select dc1.api_key,\n       concat(\'https://aurum.sensedata.io/cliente/\',dc2.id_sensedata) as url,\n       dc2.status,\n       dc2.fase,\n       Score_Calculado_v1,\n       Score_Calculado_v2,\n       Score_Calculado_v3,\n       Score_Calculado_v4,\n       Score_Calculado_v5,\nfrom df_comparison dc1\njoin df_clientes dc2 on dc1.api_key = dc2.api_key', 'SQL_DEEPNOTE_DATAFRAME_SQL', audit_sql_comment='', sql_cache_mode='cache_disabled')
df_2

# Definição de Métricas de Engajamento

## Como avaliar curvas de distribuição:

### - Distribuição Normal

### - Simetria

### - Pico central

In [ ]:
# Generate random data following a normal distribution
np.random.seed(42)
data = np.random.normal(0, 1, 1000)  # Mean: 0, Standard Deviation: 1

# Create a histogram using Plotly Express
fig = px.histogram(data, nbins=70, title="Exemplo de distribuição normal em histograma")
fig.update_layout(xaxis_title="Valor", yaxis_title="Frequência")

# Show the plot
fig.show()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=b52694be-a1af-41c9-becd-5f2f9696d2c8' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>